# Operations Room optimization
## Importing packages

In [1]:

import datetime as dt

import pandas as pd
import os.path

In [2]:
# Datum entry
datum="2021 11 24" # provide year, month, day as a script with spaces or select from the calendar
datum = dt.datetime.strptime(datum, '%Y %m %d').date()
today = dt.date.today()


if datum<today:
    print("You selected a day in the past.")


In [3]:
## inputs we can create a table or sth similar

# department ids
deps=['CH1', 'CH2', 'CH3', 'HT', 'IM1', 'IM2', 'OR1', 'OR2']

# Departments capacities
deps_cap=[660, 435, 600,260,0, 825, 930,0]



# operation room ids
op_rooms_id=["HLK1","HLK2","OP1","OP2","OP3","OP4","OP5","OP6"]

# Operation rooms capacities
op_rooms_cap=[435, 390, 555, 555 ,555,435,435,320]





# Operations basket simulation

# Simulation starts!
### An operation is registered and transfered to the basket
<br>
We assume that the Hospital has a strategy of FIFO for its services. <br>
Every single day is a large basket for the operations registered by the departments. After every operatin's registration, algoritm runs and returns a solution for the optimal allocations. That means, time schedule of the operations can be different after every run, since it looks for the optimal allocation. The hospital staff do not promise the patient for a particular time slot. The hospital promises only the date. <br> 
Once the last registrated operation causes the algoritm to hit the constraints and to fail the assigning of all operations in the basket to the operation rooms, it stops there and rejects the last registration.  The previous solution which assigns all the operations in the into the operation-rooms remains the last valid solution. <br> 


In [37]:

if os.path.isfile(f'{datum}.csv'):   
    print("it is there")
    basket=pd.read_csv(f'{datum}.csv', index_col="operation_id")
else:
    # we create an empty dataframe only first time
    basket=pd.DataFrame(columns=['Abteilungs-ID','ops','weight','value',"op_room_id"])
    basket.index.name = 'operation_id'

#output
basket.tail()

it is there


,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_7,OR1,xxxx,76.0,7.0,HLK1
op_or1_8,OR1,xxxx,76.0,7.0,HLK1
op_or1_9,IM2,xxxx,122.0,1.0,HLK1
op_or1_11,IM2,xxxx,102.0,1.0,HLK1
op_or1_19,IM2,xxxx,302.0,5.0,HLK2


In [29]:
# We populate basket with 5 cells: index, Abteilung-ID, ops, value, weight
# Every entry will be recorded in this table. 


ind="op_or1_34" # this id can be provided by the departments
basket.loc[ind,'ops']=str("xxxx") # codes of the operations to be done
basket.loc[ind,'Abteilungs-ID']=str('IM2') # name the Department who is asking for the operation
basket.loc[ind,'value']=int(5) # importance or the urgency of the operation
basket.loc[ind, 'weight']=int(302) # in minutes



In [30]:
# show last 5 entries (output)
basket.tail()

,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_7,OR1,xxxx,76.0,7.0,OP5
op_or1_8,OR1,xxxx,76.0,7.0,OP5
op_or1_9,IM2,xxxx,122.0,1.0,OP5
op_or1_11,IM2,xxxx,102.0,1.0,OP5
op_or1_19,IM2,xxxx,302.0,5.0,NaN


In [31]:
# output
basket_weight=sum(basket['weight'])
print(basket_weight)

723.0


In [32]:


#Create the data for the optimization
data = {}

w = list(basket['weight'])
v = list(basket['value'])
d= list(basket['Abteilungs-ID'])
l=list(basket.index)
items=list(zip(l,w,v,d)) # item_no, weight, value, department


data['items'] = items


data['bins'] = list(zip(op_rooms_id,op_rooms_cap))
data['bin_capacities'] = op_rooms_cap



data['deps'] = list(zip(deps,deps_cap))
data['dep_capacities'] = deps_cap


### Operation Rooms Allocation Problem Solving
### Optimization with Google OR tools
Please visit the following website for the tutorials and documentation. 

https://developers.google.com/optimization/introduction/python

In [33]:

#Normally, the algorihtm will be executed every time when an operation is tried to be registered. 
# This registration process can continue until one day or one week before the operation.
# It depends on the internal process of the hospital.
#But until it is frozen, flow of ops continues. 

from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')


# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i,w,v,d in data['items']:
    for j,c in data['bins']:
        x[(d,i, j)] = solver.IntVar(0, 1, 'x_%s_%s_%s' % (d,i, j))

#print('number of variables',len(x))

# Constraints
# Each item can be in at most one bin.
for i,w,v,d in data['items']:
    solver.Add(sum(x[(d,i, j)] for j,c in data['bins']) <= 1) # department_no, item_no, bin_no


#print('Number of constraints =', solver.NumConstraints())

# The amount packed in each bin cannot exceed its capacity.
for j,c in data['bins']:
    solver.Add(
        sum(x[(d,i, j)] * w
            for i,w,v,d in data['items']) <= c)

#print('Number of constraints =', solver.NumConstraints())

# The dapertment cannot exceed its capacity

#for k in data['deps']:
for k,u in data['deps']:
    total=0
    for i,w,v,d in data['items'] :
        for j,c in data['bins']:
            if d==k:
                total+= x[(d,i, j)]*w
    solver.Add(total <= u)

#print('Number of constraints =', solver.NumConstraints())

# Objective
objective = solver.Objective()

for i,w,v,d in data['items']:
    for j,c in data['bins']:
        objective.SetCoefficient(x[(d,i, j)], v)
objective.SetMaximization()

status = solver.Solve()
count=sum(v.solution_value() for k,v in x.items())



# Check the solution allocates all the registered operations to the op_rooms.
# If it fails, reject the this solution and this new item (operation) and continue with the previous solution. 


if basket.shape[0]>count:
    print('Sorry,',basket.tail(1).index,', we cannot offer you a time slot on that day.')
    basket.drop(basket.tail(1).index, inplace=True)

else:
    #continue
    print(basket.tail(1), "registered for the operation.")

if status == pywraplp.Solver.OPTIMAL:


    total_weight = 0

    for j,c in data['bins']:
        bin_weight = 0
        bin_value = 0

        for i,w,v,d in data['items']:
            if x[(d,i, j)].solution_value() > 0:
                #print('Item', i,'-department:', d, '- weight:', w, ' value:', v)
                bin_weight += w
                bin_value += v
                basket.loc[i,"op_room_id"]=j

        total_weight += bin_weight
    print('.........')
    print('Total planned op value:', objective.Value())    
    print('Total planned  op time:', total_weight)
    print("Basket total weight", basket["weight"].sum())
    print('Total number of ops:', count) 

    print('We will inform you about the exact time of the operation.')


else:
    print('The problem does not have an optimal solution.')
    #break
    
# write upated df back to csv 
basket.to_csv(f'{datum}.csv', index=True)

             Abteilungs-ID   ops  weight  value op_room_id
operation_id                                              
op_or1_19              IM2  xxxx   302.0    5.0        NaN registered for the operation.
.........
Total planned op value: 24.0
Total planned  op time: 723.0
Basket total weight 723.0
Total number of ops: 6.0
We will inform you about the exact time of the operation.


In [34]:
# print outputs operation room capacities
if status == pywraplp.Solver.OPTIMAL:
    print('')
    print('...Op rooms capacities')
    print(data['bins'],'\n')
    total_weight = 0

    for j,c in data['bins']:
        bin_weight = 0
        bin_value = 0
        print('...op room ', j, '\n')
        for i,w,v,d in data['items']:
            if x[(d,i, j)].solution_value() > 0:
                print('Item', i,'-department:', d, '- weight:', w, ' value:', v)
                bin_weight += w
                bin_value += v
        print('Planned op room time:', bin_weight)
        print('Planned op value:', bin_value)
        print()
        total_weight += bin_weight
    print('.........')
    print('Total planned op value:', objective.Value())    
    print('Total planned op time:', total_weight)
    print('Total number of ops:', sum(v.solution_value() for k,v in x.items()))


...Op rooms capacities
[('HLK1', 435), ('HLK2', 390), ('OP1', 555), ('OP2', 555), ('OP3', 555), ('OP4', 435), ('OP5', 435), ('OP6', 320)] 

...op room  HLK1 

Item op_or1_1 -department: OR1 - weight: 45.0  value: 3.0
Item op_or1_7 -department: OR1 - weight: 76.0  value: 7.0
Item op_or1_8 -department: OR1 - weight: 76.0  value: 7.0
Item op_or1_9 -department: IM2 - weight: 122.0  value: 1.0
Item op_or1_11 -department: IM2 - weight: 102.0  value: 1.0
Planned op room time: 421.0
Planned op value: 19.0

...op room  HLK2 

Item op_or1_19 -department: IM2 - weight: 302.0  value: 5.0
Planned op room time: 302.0
Planned op value: 5.0

...op room  OP1 

Planned op room time: 0
Planned op value: 0

...op room  OP2 

Planned op room time: 0
Planned op value: 0

...op room  OP3 

Planned op room time: 0
Planned op value: 0

...op room  OP4 

Planned op room time: 0
Planned op value: 0

...op room  OP5 

Planned op room time: 0
Planned op value: 0

...op room  OP6 

Planned op room time: 0
Planned 

In [35]:
# print outputs for departments
print('dep capacities', data['deps'])
print('.....')
for dep, dep_cap in data['deps']:
    total_weight=0
    #print()
    for op_room, op_room_cap in data['bins']:
        for item,weight,value,department in data['items']:
            if dep==department and x[(department,item,op_room)].solution_value()>0:
                
                total_weight+=weight
    print("Department", dep,":total weight", total_weight,"\n")


dep capacities [('CH1', 660), ('CH2', 435), ('CH3', 600), ('HT', 260), ('IM1', 0), ('IM2', 825), ('OR1', 930), ('OR2', 0)]
.....
Department CH1 :total weight 0 

Department CH2 :total weight 0 

Department CH3 :total weight 0 

Department HT :total weight 0 

Department IM1 :total weight 0 

Department IM2 :total weight 526.0 

Department OR1 :total weight 197.0 

Department OR2 :total weight 0 



In [36]:
basket.tail()

,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_7,OR1,xxxx,76.0,7.0,HLK1
op_or1_8,OR1,xxxx,76.0,7.0,HLK1
op_or1_9,IM2,xxxx,122.0,1.0,HLK1
op_or1_11,IM2,xxxx,102.0,1.0,HLK1
op_or1_19,IM2,xxxx,302.0,5.0,HLK2
